In [19]:
import pandas as pd
# import numpy as np
# import re

import matcher  
# import spacy
import matcher2


In [102]:
from fuzzywuzzy import fuzz


In [178]:
fuzz.ratio("Esme Sureface Liquid Foundation SPF 20 /PA++ Natural 30g - Alas Bedak".lower().strip(),"Fit Me Fresh Tint SPF50".lower().strip())

30

In [121]:
fuzz.partial_token_sort_ratio('aa','a')

100

In [22]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

# Our sentences to encode
sentences = [
    "This framework generates embeddings for each input sentence",
    "Sentences are passed as a list of string.",
    "The quick brown fox jumps over the lazy dog."
]

# Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

# Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

Sentence: This framework generates embeddings for each input sentence
Embedding: [-1.37173282e-02 -4.28515524e-02 -1.56285744e-02  1.40537396e-02
  3.95537578e-02  1.21796273e-01  2.94333789e-02 -3.17523926e-02
  3.54959629e-02 -7.93139860e-02  1.75878331e-02 -4.04369682e-02
  4.97259609e-02  2.54912544e-02 -7.18700513e-02  8.14968869e-02
  1.47067406e-03  4.79627252e-02 -4.50336449e-02 -9.92174745e-02
 -2.81769522e-02  6.45046309e-02  4.44670394e-02 -4.76217046e-02
 -3.52952555e-02  4.38671894e-02 -5.28565869e-02  4.33016277e-04
  1.01921484e-01  1.64072458e-02  3.26996297e-02 -3.45987044e-02
  1.21339448e-02  7.94870928e-02  4.58343793e-03  1.57778151e-02
 -9.68207140e-03  2.87625752e-02 -5.05806245e-02 -1.55793792e-02
 -2.87906956e-02 -9.62282438e-03  3.15556824e-02  2.27348953e-02
  8.71449336e-02 -3.85027155e-02 -8.84718820e-02 -8.75501242e-03
 -2.12343279e-02  2.08923873e-02 -9.02077779e-02 -5.25732338e-02
 -1.05638886e-02  2.88310777e-02 -1.61454976e-02  6.17838465e-03
 -1.23234

In [11]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)

Sentence embeddings:
tensor([[ 6.7657e-02,  6.3496e-02,  4.8713e-02,  7.9305e-02,  3.7448e-02,
          2.6528e-03,  3.9375e-02, -7.0984e-03,  5.9361e-02,  3.1537e-02,
          6.0098e-02, -5.2905e-02,  4.0607e-02, -2.5931e-02,  2.9843e-02,
          1.1269e-03,  7.3515e-02, -5.0382e-02, -1.2239e-01,  2.3703e-02,
          2.9727e-02,  4.2477e-02,  2.5634e-02,  1.9952e-03, -5.6919e-02,
         -2.7160e-02, -3.2904e-02,  6.6025e-02,  1.1901e-01, -4.5879e-02,
         -7.2621e-02, -3.2584e-02,  5.2341e-02,  4.5055e-02,  8.2530e-03,
          3.6702e-02, -1.3942e-02,  6.5392e-02, -2.6427e-02,  2.0637e-04,
         -1.3664e-02, -3.6281e-02, -1.9504e-02, -2.8974e-02,  3.9427e-02,
         -8.8409e-02,  2.6243e-03,  1.3671e-02,  4.8306e-02, -3.1157e-02,
         -1.1733e-01, -5.1169e-02, -8.8529e-02, -2.1896e-02,  1.4299e-02,
          4.4417e-02, -1.3482e-02,  7.4339e-02,  2.6638e-02, -1.9876e-02,
          1.7919e-02, -1.0605e-02, -9.0426e-02,  2.1327e-02,  1.4120e-01,
         -6.4718e

In [13]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Función de Pooling Promedio - Toma en cuenta la máscara de atención para un promedio correcto
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state  # El último estado oculto contiene todas las incrustaciones de tokens
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Frases para las cuales queremos obtener incrustaciones
# sentences = ['hola', 'hello', 'hola']
sentences = [
    "Samsung S14",
    "Iphone 11",
]

# Cargar tokenizador y modelo de HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenizar las frases
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Calcular incrustaciones de tokens
with torch.no_grad():
    model_output = model(**encoded_input)

# Realizar Pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalizar las incrustaciones
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

# print("Sentence embeddings:")
# print(sentence_embeddings)

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular la similitud del coseno entre las incrustaciones
cosine_similarities = cosine_similarity(sentence_embeddings, sentence_embeddings)

# Escalar los valores de similitud del coseno al rango de 1 a 100
scaled_similarity = cosine_similarities * 100

# Redondear los valores para obtener un resultado del 1 al 100
result = scaled_similarity.round()

print("Resultado del 1 al 100:")
print(result)

Resultado del 1 al 100:
[[100.  42.]
 [ 42. 100.]]


In [75]:
cosine_similarities

array([[1.0000004, 0.7517575],
       [0.7517575, 1.       ]], dtype=float32)

In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("sentence-similarity", model="sentence-transformers/all-MiniLM-L6-v2")

KeyError: "Unknown task sentence-similarity, available tasks are ['audio-classification', 'automatic-speech-recognition', 'conversational', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction', 'image-segmentation', 'image-to-image', 'image-to-text', 'mask-generation', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification', 'translation', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']"

In [2]:
matcher2.harmony_datum_matcher('prueba2',['title','product_name'])

[[0, 5], [0, 24], [0, 27], [0, 29], [0, 33], [0, 87], [0, 88], [0, 90], [0, 92], [0, 93], [0, 94], [0, 95], [0, 117], [0, 118], [0, 128], [0, 139], [0, 140], [0, 141], [0, 142], [0, 143], [0, 144], [0, 145], [0, 146], [0, 195], [0, 197], [0, 205], [0, 206], [0, 209], [0, 211], [0, 213], [0, 218], [0, 219], [0, 256], [0, 258], [0, 260], [0, 262], [0, 267], [0, 281], [0, 284], [0, 286], [0, 287], [0, 288], [0, 293], [0, 294], [0, 295], [0, 296], [1, 0], [1, 1], [1, 2], [1, 3], [1, 12], [1, 13], [1, 14], [1, 16], [1, 17], [1, 18], [1, 20], [1, 21], [1, 22], [1, 26], [1, 28], [1, 30], [1, 31], [1, 38], [1, 39], [1, 40], [1, 41], [1, 42], [1, 43], [1, 44], [1, 45], [1, 46], [1, 47], [1, 48], [1, 49], [1, 50], [1, 51], [1, 52], [1, 53], [1, 54], [1, 55], [1, 56], [1, 57], [1, 58], [1, 59], [1, 60], [1, 61], [1, 62], [1, 63], [1, 64], [1, 65], [1, 66], [1, 67], [1, 68], [1, 69], [1, 70], [1, 71], [1, 72], [1, 76], [1, 77], [1, 80], [1, 82], [1, 83], [1, 93], [1, 100], [1, 101], [1, 103], [1, 

In [ ]:
print(matcher2.get_df('prueba2/result_withDuplicates_simple.json').filter(like='EWA',axis=0))

In [39]:
series_origin, series_origin_x=matcher2.get_df('prueba2/origin.json')['title'].astype('str'), matcher2.get_df('prueba2/origin.json')['link'].astype('str')

In [48]:
series_alternative, series_alternative_x=matcher2.get_df('prueba2/alternative_2.json')['product_name'].astype('str'), matcher2.get_df('prueba2/alternative_2.json')['url'].astype('str')

In [19]:
from fuzzywuzzy import fuzz
precision = 80

In [49]:
nr = []
for index_row_origin, index_row_origin_x in zip(series_origin, series_origin_x):
    for index_row_alternative, index_row_alternative_x in zip(series_alternative, series_alternative_x):
        if index_row_alternative == 'nan':
            continue
        radio = fuzz.ratio(index_row_origin, index_row_alternative )
        if radio in range(50,79) :
            nr.append({'similitud':radio, 'nombre_origin':index_row_origin,'link_origin': index_row_origin_x,'nombre_alternative':index_row_alternative, 'link_alternative': index_row_alternative_x})
            # print(f'{radio} #-# {index_row_origin} #-# {index_row_alternative}')

In [50]:
pd.DataFrame(nr).to_csv('shopee_lazada_range.csv')

In [ ]:
matcher2.get_df('shopee_lazada_range.csv').drop_duplicates()

In [28]:
# Cargar los datos del archivo CSV
df = matcher.get_df('colores.csv')

# nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("fr_core_news_sm")
# nlp = spacy.load("es_core_news_sm")

In [15]:
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

In [ ]:
lemmatized_text = " ".join([token.lemma_ for token in df])
doc = nlp("naranja")
for token in doc:
    print(token.text, token.lemma_)

In [71]:
df["word"] = df["word"].map(lambda x: x.lower())
df["lemmatized_text"] = df["word"].apply(lemmatize_text)
df["tag"] = "color"
df["language"] = "fr"

In [29]:
unique_values_by_column = df.apply(lambda col: col.astype(str).unique())

In [42]:
from sqlalchemy import create_engine


In [ ]:
# Establecer una conexión con la base de datos SQLite (si no existe, la creará)
engine = create_engine('sqlite:///data.sqlite', echo=True)

# Guardar el DataFrame en la base de datos (reemplazando si ya existe)
df.to_sql('LemmatizedKeywords', con=engine, if_exists='append', index=False)

In [ ]:
# Consultar la tabla para verificar si se ha cargado correctamente
consulta = pd.read_sql_query('SELECT * FROM LemmatizedKeywords', engine)
print(consulta)

In [66]:
consulta.to_csv("respaldo.csv")

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Example product data from two links
product_a = {    "description": "Pink printed Kurta with Trousers with dupatta\n\nKurta design:\nStriped\nA-line shape\nEmpire style\nV-neck, three-quarter regular sleeves\nThread work detail\nCalf length with flared hem\nSilk blend machine weave fabric\n\nTrousers design:\nSolid Trousers\nPartially elasticated waistband\nSlip-on closure\n\nSize & Fit\nThe model (height 5'8) is wearing a size S\n\nMaterial & Care\nTop-Silk Blend\nBottom-Shantoon\nDupatta-Silk Blend\nDry Clean\n\nSpecifications\nSleeve Length\nThree-Quarter Sleeves\nTop Shape\nA-Line\nTop Type\nKurta\nBottom Type\nTrousers\nDupatta\nWith Dupatta\nTop Pattern\nStriped\nTop Design Styling\nEmpire\nTop Hemline\nFlared\nTop Length\nCalf Length\nNeck\nV-Neck\nPrint or Pattern Type\nStriped\nBottom Pattern\nSolid\nBottom Closure\nSlip-On\nWaistband\nPartially Elasticated\nOccasion\nFestive\nOrnamentation\nThread Work\nWeave Pattern\nRegular\nWeave Type\nMachine Weave\nPattern Coverage\nLarge\nDupatta Pattern\nDyed\nDupatta Border\nPrinted\nNumber of Items\n3",
}
product_b = {    "description": """Color may slightly vary due to photography\nBottomwear fabric: shantoon\nDupatta length: 2 m\nDupatta fabric: silk blend\nPackage contains: 1 kurta, 1 dupatta, 1 pants\nKurta, Bottomwear & Dupatta\nDry clean\nMuslin silk\nProduct Code: 466764916002\nMRP: Rs. 7,499.00 inclusive of all taxes(MRP changes as per size selection)\nNet Qty: 1N\nMarketed By: FASHOR LIFESTYLE PVT LTD, 28/30,Sudershan pura, Industrial Area, 22 Godam, Jaipur, Rajasthan, 302006\nManufactured By: FASHOR LIFESTYLE PVT LTD, 28/30,Sudershan pura, Industrial Area, 22 Godam, Jaipur, Rajasthan, 302006\nCountry Of Origin: India\nCustomer Care Address: Tower-B, 7th Floor,IBC Knowledge Park,Bannerghatta Main Road,Bhavani Nagar, S.G. Palya,Bengaluru, Karnataka – 560029,Ph: 1800-889-9991,E-mail: customercare@ajio.com\nCommodity: Women's Kurta Suit Sets",
"""}
# Preprocess and vectorize the data
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([product_a['description'], product_b['description']])

# Calculate similarity
similarity_score = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

# Determine if products are a match based on a threshold
threshold = 0.8  # This is an example threshold
print(similarity_score)
if similarity_score[0][0] > threshold:
    print('Products are a match')
else:
    print('Products are not a match')

[[0.12283303]]
Products are not a match


In [2]:
import matcher

In [3]:
def toolkit_detect_language(file_path: str = None, typedetect: str = None):
    from transformers import pipeline

    pipe = pipeline("text-classification", model="ImranzamanML/GEFS-language-detector")

    
    def dividir_texto(texto, longitud_maxima):
        palabras = texto.split()  # Dividir el texto en palabras
        palabras_seleccionadas = []

        longitud_actual = 0
        for palabra in palabras:
            if longitud_actual + len(palabra) + sum(map(len, palabras_seleccionadas)) <= longitud_maxima:
                palabras_seleccionadas.append(palabra)
                longitud_actual += len(palabra)
            else:
                break
            
        return ' '.join(palabras_seleccionadas)
    
    longitud_maxima = 2250
    
    df = matcher.get_df("prueba2\origin.json")

    text_combined = ' '.join(df.apply(lambda row: ', '.join(row.astype(str).unique()), axis=1))
    texto_cortado = dividir_texto(text_combined, longitud_maxima)
    
    return pipe(texto_cortado)[0]['label']

toolkit_detect_language()

'en'

In [59]:
matcher.detect_language_file('files/farmatodo.json')

'The detected language in the CSV is: es'

In [41]:
import os
import numpy as np

In [92]:
def Matcherito(file_path_x: str = None, file_path_y: str = None, typematch: str = None, queries: list = [], language:dict = {'all':None, 'columns':{}}, datatype: str = None, algorithmatch:str=None) -> pd.DataFrame:
    
    
    extensions = ['csv', 'json']

    if not any(file_path_x.endswith(extension) for extension in extensions) or not any(file_path_y.endswith(extension) for extension in extensions):
        raise ValueError("The file does not have a valid extension.")

    if file_path_x is None:
        raise ValueError("required field 'file_path_x'")
    elif file_path_y is None:
        raise ValueError("required field 'file_path_y'")
    else:
        if file_path_x == file_path_y:
            raise ValueError("Cannot put the same file on both paths")
        if not os.path.exists(file_path_x) or not os.path.exists(file_path_y):
            raise FileNotFoundError("required file does not exist in the path")

    if typematch is None:
        raise ValueError("required field 'typematch'")
    elif not typematch in ['exact', 'similar']:#cambiar a variable de entorno con la lista de tipo de match
        raise ValueError("match type not valid")
    
    # if not language in ['es', 'en', 'fr']:
        # raise ValueError("unsupported language")

    if type(file_path_x) != str or type(file_path_y) != str or type(typematch) != str or type(queries) != list or type(language) != dict:
        raise TypeError("Data type error in one of the attributes")
    

    def language_input(language:dict = {'all':None, 'columns':{}}):
        if not language:
            raise ValueError('Please enter the language')
        if 'all' not in language and language not in ['es', 'en', 'fr']:
            raise ValueError("unsupported language")
        else:
            print(f'idioma de archivo: {language.all}')
            print('ola')


    #start logic for match
    if len(queries) == 0:
        language_input(language)
        # if not language['all'] is  None:
            # print('hola')
            # pass #detect language of file
            


        pass #run script auto
    else:
        pass # run script manual

        # Perform data matching based on the query
        if np.ndim(queries) == 1:
            # Single query case
            pass
        else:
            # Multiple queries case
            pass

In [93]:
Matcherito('colores.csv','respaldo.csv', 'exact', language={'columns':{'name':'es'}})

ValueError: unsupported language

In [77]:
dics = {}
len(dics)

0

In [112]:

# Example usage
language = {'all': 'en'}

if not language:
    raise ValueError('Please enter the language')

if 'all' in language:
    all_value = language['all']
    if all_value is not None or all_value not  in ['es', 'en', 'fr']:
        raise ValueError('The entered language is not valid')
    else:
        print(all_value)

# # Check if the language is in a list of allowed languages
# if 'all' not in language and language['all'] not in ['es', 'en', 'fr']:
#     raise ValueError('The entered language is not valid')

ValueError: The entered language is not valid